In [92]:
from jax.experimental import sparse
import jax.numpy as jnp
import numpy as np
from sojo.pstabilizer import group_instructorss_by_qubits, char_to_index, Instructor
from sojo.pstabilizer import weightss_to_lambda, word_to_index, generate_pauli_combination, instructor_to_lut
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
ins = Instructor(4)
ins.append("h", 0)
ins.append("rx", 1, 0.78)
ins.append("h", 2)
ins.append("h", 0)
ins.append("cx", [0, 1])
ins.append("h", 2)
ins.append("h", 2)
ins.append("ry", 0, 0.56)
ins.append("cx", [1, 2])
ins.append("h", 1)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 1)
ins.append("h", 2)
ins.append("h", 3)
ins.append("h", 0)
ins.append("h", 2)
ins.append("cx", [1, 3])
ins.clustering()

In [93]:
num_qubits = 3
ins = Instructor(num_qubits)
for k in range(5):
	for i in range(num_qubits - 1):
		ins.append('cx', [i, i + 1])
	ins.append('cx', [num_qubits - 1, 0])
	for i in range(num_qubits):
		ins.append('rx', i, np.random.rand())
		ins.append('ry', i, np.random.rand())
		ins.append('rz', i, np.random.rand())

ins.clustering()

In [94]:
LUT = instructor_to_lut(ins)

(5, 3, 4, 4)


In [88]:
num_qubits = 3
term = generate_pauli_combination(num_qubits)

k = 0
# Init
# 2**n is th
lambdas = np.ones(4**num_qubits)

weightss = []
for w, word in enumerate(term):
    weights = []
    for j, char in enumerate(word):
        index = char_to_index(char)
        weights.append(LUT[k][j][index])
    weightss.append(weights)
weightss = np.array(weightss)
lambdas = weightss_to_lambda(weightss, lambdas)

In [ ]:
non_zero_indices = np.nonzero(lambdas)
non_zero_values = np.array(lambdas)[non_zero_indices]
non_zero_indices, non_zero_values
print(non_zero_indices)
print(non_zero_values)

[ 1.00000000e+00  1.26372584e-01  1.72296422e+00  1.24194415e-01
  7.72793255e-01  9.76598807e-02  1.33149513e+00  9.59766059e-02
  1.53982223e+00  1.94591314e-01  2.65305861e+00  1.91237320e-01
 -1.78151899e-01 -2.25135158e-02 -3.06949348e-01 -2.21254708e-02
  1.49805481e+00  1.89313058e-01  2.58109484e+00  1.86050040e-01
  1.15768665e+00  1.46299854e-01  1.99465269e+00  1.43778216e-01
  2.30673809e+00  2.91508454e-01  3.97442721e+00  2.86483987e-01
 -2.66881309e-01 -3.37264807e-02 -4.59826947e-01 -3.31451679e-02
  8.69150135e-01  1.09836749e-01  1.49751459e+00  1.07943592e-01
  6.71673362e-01  8.48810985e-02  1.15726917e+00  8.34180800e-02
  1.33833670e+00  1.69129067e-01  2.30590625e+00  1.66213942e-01
 -1.54840747e-01 -1.95676253e-02 -2.66785067e-01 -1.92303559e-02
  2.02442085e-02  2.55831294e-03  3.48800470e-02  2.51421762e-03
  1.56445878e-02  1.97704699e-03  2.69550651e-02  1.94297042e-03
  3.11724822e-02  3.93934713e-03  5.37090716e-02  3.87144818e-03
 -3.60654418e-03 -4.55768